## MARKDOWN

In [2]:
import pandas as pd
from pymining import seqmining
import numpy as np
from scipy.special import entr
from collections import defaultdict
import itertools

In [3]:
dffa = pd.read_pickle("../code/data/786/786_mpframe_trajprocessed.p")
dffa.shape

(98993, 10)

In [22]:
trajs_combined_balanced = pd.read_pickle("../data/786/786_trajs_combined_balanced.p")

In [25]:
area = getuniqueareas(trajs_combined_balanced.traj)
trajs_combined_balanced['traj'] = trajs_combined_balanced.apply(lambda x: add_temporal_sign(x, area), axis=1)
seqs = trajs_combined_balanced.apply(lambda x: (x['traj'], x['revisit_intention']), axis=1)
freq_seq_enum(seqs, 200)

{(('1f-inner-long',), 425, 273.0),
 (('1f-inner-medium',), 1150, 632.0),
 (('1f-inner-medium', '1f-left-medium'), 304, 182.0),
 (('1f-inner-medium', '1f-right-medium'), 283, 162.0),
 (('1f-inner-short',), 534, 264.0),
 (('1f-left-long',), 381, 250.0),
 (('1f-left-medium',), 1173, 688.0),
 (('1f-left-medium', '1f-inner-medium'), 348, 202.0),
 (('1f-left-medium', '1f-right-medium'), 378, 207.0),
 (('1f-left-short',), 589, 316.0),
 (('1f-long',), 1560, 968.0),
 (('1f-long', '1f-inner-long'), 385, 246.0),
 (('1f-long', '1f-inner-medium'), 456, 282.0),
 (('1f-long', '1f-left-long'), 345, 231.0),
 (('1f-long', '1f-left-medium'), 475, 303.0),
 (('1f-long', '1f-right-long'), 563, 360.0),
 (('1f-long', '1f-right-medium'), 511, 330.0),
 (('1f-long', '2f-left-medium'), 205, 123.0),
 (('1f-long', '2f-long'), 288, 188.0),
 (('1f-long', '2f-medium'), 330, 195.0),
 (('1f-long', '2f-right-medium'), 231, 145.0),
 (('1f-medium',), 3274, 1649.0),
 (('1f-medium', '1f-inner-medium'), 650, 335.0),
 (('1f-me

In [14]:
### 이 코드에서 계산중에 같이 collect할 수 있을거라 생각했는데 , 포기.
from collections import defaultdict

def freq_seq_enum(sequences, min_support):
    '''Enumerates all frequent sequences.

       :param sequences: A sequence of sequences.
       :param min_support: The minimal support of a set to be included.
       :rtype: A set of (frequent_sequence, support).
    '''
    freq_seqs = set()
    _freq_seq(sequences, tuple(), 0, 0, min_support, freq_seqs)
    return freq_seqs


def _freq_seq(sdb, prefix, prefix_support, revisit_support, min_support, freq_seqs):
    if prefix:
#         print('prefix: yes', prefix)
        freq_seqs.add((prefix, prefix_support, revisit_support))
#         print('freq seqs', freq_seqs)
    locally_frequents = _local_freq_items(sdb, prefix, min_support)
#     print('locally frequents', locally_frequents)
    if not locally_frequents:
#         print ('not locally frequents')
        return
    for (item, support1, support2) in locally_frequents:
        new_prefix = prefix + (item,)
        new_sdb = _project(sdb, new_prefix)
#         print('new_prefix', new_prefix)
#         print('new_sdb', new_sdb)
        _freq_seq(new_sdb, new_prefix, support1, support2, min_support, freq_seqs)


def _local_freq_items(sdb, prefix, min_support):
    items = defaultdict(int)  ## for support
    items2 = defaultdict(int)  ## for revisit_intention_support
    freq_items = []
    for entry in sdb:
        visited = set()
        for element in entry[0]:
#             print (element)
            if element not in visited:
                items[element] += 1
                items2[element] += entry[1]
                visited.add(element)
    # Sorted is optional. Just useful for debugging for now.
    for item in items:
        support = items[item] ## support
        support2 = items2[item] ## revisit_intention
        if support >= min_support:
            freq_items.append((item, support, support2))
    return freq_items


def _project(sdb, prefix):
    new_sdb = []
    if not prefix:
        return sdb
    current_prefix_item = prefix[-1]
    for entry in sdb:
        j = 0
        projection = None
        for item in entry[0]:
            if item == current_prefix_item:
                projection = (entry[0][j + 1:], entry[1])
                break
            j += 1
        if projection:
            new_sdb.append(projection)
    return new_sdb

In [6]:
def puttogether(a, b):
    if b == 0:
        c = 'zero'
    elif b < 60:
        c = 'short'
    elif b < 600:
        c = 'medium'
    else:
        c = 'long'
    return '-'.join([a,c])

def add_temporal_sign(x, area):  # Make features like 1f-medium, or 1f-inner-short
    idcs = [i for i, y in enumerate(x['traj']) if y in area]
    zt = [puttogether(x['traj'][index], x['dwell_time'][index]) for index in idcs] 
    return zt

def getuniqueareas(trajseries):
    aggregated_traj = list(itertools.chain.from_iterable(trajseries))
    uniqueareas = sorted(list(set(aggregated_traj)))
    return uniqueareas

In [7]:
'''
Name: sequencefeaturegenerator2.py
Date: 2016-10-25
Description: Frequent Sequence 를 찾은 후 Feature로 이용하는 모듈 (faster ver)


Input: 
TO DO: 
'''

import pandas as pd
import numpy as np
from scipy.special import entr
from collections import defaultdict


def is_subseq(x, y):
    it = iter(y)
    return all(c in it for c in x)

# assert is_subseq('india', 'indonesia')
# assert is_subseq('oman', 'romania')
# assert is_subseq('mali', 'malawi')
# assert is_subseq((''), ('a', 'b', 'c', 'd', 'e'))
# assert not is_subseq('mali', 'banana')
# assert not is_subseq('ais', 'indonesia')
# assert not is_subseq('ca', 'abc')

def entropy(prob1, prob2):
    return(-prob1*np.log2(prob1)-prob2*np.log2(prob2))

def informationGain(a, b, c, d):
    ''' 
    a = (True, 1.0) - Subsequence, Revisit intention      
    b = (True, 0.0)       
    c = (False, 1.0)    
    d = (False, 0.0) 
    '''
    # Entropy before
    prob1a = (a+c) / (a+b+c+d)
    prob2a = 1-prob1a
    entropy_before = entropy(prob1a, prob2a)
    
    # Entropy after sequence
    prob1b = a / (a+b)
    prob2b = 1 - prob1b
    prob3b = c / (c+d)
    prob4b = 1 - prob3b
    
    entropy1 = entropy(prob1b, prob2b)
    entropy2 = entropy(prob3b, prob4b)
    entropy_after = (a+b)/(a+b+c+d)*entropy1 + (c+d)/(a+b+c+d)*entropy2
    
    IG = entropy_before-entropy_after
    
    return IG



def recursivelyFindLongestSequence(aabaaba, new_list):

    try:
        for item in aabaaba:
            testval = 0
            for longt in new_list:
                testval += is_subseq(item, longt)

            if testval == 0:
                new_list.append(item)

        for item in new_list:
            aabaaba.remove(item)


        recursivelyFindLongestSequence(aabaaba, new_list)
    except:
        pass


### If multiple subsequences have same support, there is a chance to have closed sequence and its subsequence
### Here, we only select independent longest sequences by deleting its subs.
### Technique used: Chance key, value and removing subs by dynamic programming
### Method recursivelyFindLongestSequence is used for the dynamic programming part.
### Input: freq_seqs_sample - Output: freqfreqfreq

def leavelongest_samesupport(freq_seqs_sample):

    freq_seqs_sample2 = {}
    for kv in freq_seqs_sample:
        freq_seqs_sample2.setdefault((kv[1], kv[2]), []).append(kv[0])

    freqfreqfreq = []


    for k, v in freq_seqs_sample2.items():
        if len(v) > 1:
            v = sorted(v, key = len, reverse=True)
            new_list = []
            new_list.append(v[0])
            recursivelyFindLongestSequence(v, new_list) 
            for item in new_list:
                freqfreqfreq.append(tuple((item, k)))
        else:
            freqfreqfreq.append(tuple((v[0], k)))

    freqfreqfreq = sorted(freqfreqfreq, key=lambda tup: tup[1], reverse=True)

    return freqfreqfreq




def generate_sortE(df, supportRatio):

    ### Extract subsequences with support greater than 200 (from 8886 for this example ~ 2.5%)
    seqs = df.apply(lambda x: (x['traj'], x['revisit_intention']), axis=1)


    freq_seqs = freq_seq_enum(seqs, seqs.shape[0]*supportRatio)   #0.025 = support ratio



    ### Minimum subsequence length == 4  &  sort by support (TODO: parameter)
    freq_seqs_sample = []
    for x in freq_seqs:
        if (len(x[0]) >= 4):
    #         if (x[0][0] != 'out') & (x[0][0] != 'in'):
            freq_seqs_sample.append(x)
    freq_seqs_sample = sorted(freq_seqs_sample, key=lambda tup: tup[1], reverse=True)   


    ### Leave longest supersequence 
    longest_sequences_support = leavelongest_samesupport(freq_seqs_sample)


    num1 = df.revisit_intention.value_counts().loc[1]
    num0 = df.revisit_intention.value_counts().loc[0]

    ### Calculate information gain easily
    ''' 
    a = (True, 1.0) - Subsequence, Revisit intention      
    b = (True, 0.0)       
    c = (False, 1.0)    
    d = (False, 0.0) 
    '''
    longest_sequences_support2 = []
    for i in longest_sequences_support:
        z = []  
        a = i[1][1]  
        b = i[1][0] - i[1][1]
        c = num1 - a
        d = num0 - b
        z.append(a)
        z.append(b)
        z.append(c)
        z.append(d)
        ig = informationGain(a, b, c, d)
        z.append(ig)
        longest_sequences_support2.append((i[0], z))


    igdict = {}

    for traj in longest_sequences_support2:
        igdict[traj[0]] = traj[1]

    # print('ig calculation has been done')

    ### SortE: Tuples list of sequences(key) and their information gain
    sortE = sorted(igdict.items(), key=lambda value: value[1][-1], reverse=True)
    return sortE


def generate_seqE(sortE, numFeatures):
    seqE = []
    for item in sortE[:numFeatures]:
        seqE.append(item[0])
    return seqE



# Sequence feature를 걍 숫자로 표현 - 해당 feature를 가지면 1, 가지지 않으면 0
def relatedfeatures(traj, seqE):
    sss = 2001
    ddd = []
    for seq in seqE:
        if is_subseq(seq, traj) == True:
            ddd.append(sss)
        sss += 1
    return ddd

def generateIGFeatureColumns(df, seqE):
    sss = 2001
    for seq in seqE:
        df[sss] = 0
        sss += 1
    for row in df.iterrows():
        for seq_ig in row[1]['seq_ig_ft']:
            df.set_value(row[0], seq_ig, 1) 


def add_frequent_sequence_features(df, supportRatio, featureRatio, temporal):
    if temporal == True:
        print('Generating feature: By considering dwell_time of each area')
        area = getuniqueareas(df.traj)
        # print(area)
        df['traj'] = df.apply(lambda x: add_temporal_sign(x, area), axis=1)
    else:
        print('Generating feature: Not considering dwell_time of each area')

    sortE = generate_sortE(df, supportRatio)
    print(sortE[:30])
    numFeatures = int(round(df.shape[0]*featureRatio))
    seqE = generate_seqE(sortE, numFeatures)
    newdf = df
    newdf['seq_ig_ft'] = df.apply(lambda x: relatedfeatures(x['traj'], seqE), axis=1)
    generateIGFeatureColumns(newdf, seqE)
    del newdf['seq_ig_ft']
    return newdf

In [12]:
add_frequent_sequence_features(trajs_combined_balanced, 0.02, 0.02, True)

Generating feature: By considering dwell_time of each area
[(('out-long', 'in-long', '1f-long', '1f-right-long'), [360.0, 203.0, 4083.0, 4240.0, 0.0038420824784204388]), (('out-long', 'in-long', '1f-long', '1f-right-medium'), [330.0, 181.0, 4113.0, 4262.0, 0.0037938349956320172]), (('out-long', 'in-long', '2f-long', '1f-long'), [151.0, 56.0, 4292.0, 4387.0, 0.0037597694192437192]), (('out-long', 'in-long', '1f-long', '1f-left-long'), [231.0, 114.0, 4212.0, 4329.0, 0.003415882100584855]), (('out-long', 'in-long', '1f-long', '1f-left-medium'), [303.0, 172.0, 4140.0, 4271.0, 0.0031368246425990121]), (('out-long', 'in-long', '2f-long', '1f-medium'), [178.0, 88.0, 4265.0, 4355.0, 0.0025977098786093]), (('out-long', 'in-long', '2f-long', '1f-right-medium'), [196.0, 101.0, 4247.0, 4342.0, 0.0025959588994602933]), (('out-long', 'in-long', '1f-long', '1f-inner-long'), [246.0, 139.0, 4197.0, 4304.0, 0.0025554542831655036]), (('out-long', 'in-long', '1f-right-long', '1f-left-long'), [134.0, 61.0,

,logs,traj,ts,dwell_time,hour_start,time_start,ts_end,hour_end,device_id,date,...,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043
16767_25e9517c47319ab64486403f94348dd6,"[3145135, 3145085, 3145074, 3145073, 3145064]","[out-medium, in-medium, 2f-medium, 2f-inner-me...","[1448711070, 1448711259, 1448711286, 144871128...","[314, 125, 98, 98, 65]","[20, 20, 20, 20, 20]","[20:44:30, 20:47:39, 20:48:06, 20:48:06, 20:48...","[1448711384, 1448711384, 1448711384, 144871138...","[20, 20, 20, 20, 20]",25e9517c47319ab64486403f94348dd6,16767.0,...,0,0,0,0,0,0,0,0,0,0
16955_53f0a6276486cc80405cf534d913fcbb,"[975012, 975011, 975010, 974986, 974983]","[out-medium, in-medium, 1f-medium, 1f-right-me...","[1464939171, 1464939171, 1464939171, 146493923...","[266, 263, 263, 198, 156]","[16, 16, 16, 16, 16]","[16:32:51, 16:32:51, 16:32:51, 16:33:54, 16:33...","[1464939437, 1464939434, 1464939434, 146493943...","[16, 16, 16, 16, 16]",53f0a6276486cc80405cf534d913fcbb,16955.0,...,0,1,0,0,0,0,0,0,0,0
16734_5b37d580c76a8c8c245cad3418349acb,"[3519755, 3519754, 3519753]","[out-long, in-medium, 1f-medium]","[1445856050, 1445856052, 1445856052]","[662, 527, 527]","[19, 19, 19]","[19:40:50, 19:40:52, 19:40:52]","[1445856712, 1445856579, 1445856579]","[19, 19, 19]",5b37d580c76a8c8c245cad3418349acb,16734.0,...,0,0,0,0,0,0,0,0,0,0
16736_70968e4fb8894b74997f6f4e89b32bee,"[3498733, 3498732, 3498730]","[out-medium, in-medium, 1f-medium]","[1446031391, 1446031392, 1446031392]","[152, 144, 144]","[20, 20, 20]","[20:23:11, 20:23:12, 20:23:12]","[1446031543, 1446031536, 1446031536]","[20, 20, 20]",70968e4fb8894b74997f6f4e89b32bee,16736.0,...,0,0,0,0,0,0,0,0,0,0
16847_e0b87d079ff087efd100c423d88d0fbd,"[2235511, 2235481, 2235479]","[out-medium, in-medium, 1f-medium]","[1455604283, 1455604408, 1455604408]","[369, 94, 94]","[15, 15, 15]","[15:31:23, 15:33:28, 15:33:28]","[1455604652, 1455604502, 1455604502]","[15, 15, 15]",e0b87d079ff087efd100c423d88d0fbd,16847.0,...,0,0,0,0,0,0,0,0,0,0
16706_3352c133a2ee0a29ed3351ab69dae032,"[3822083, 3822044, 3822043, 3822038, 3822037, ...","[out-long, in-long, 3f-medium, 2f-right-medium...","[1443421497, 1443421642, 1443421642, 144342166...","[1805, 837, 499, 441, 481, 593, 465]","[15, 15, 15, 15, 15, 15, 15]","[15:24:57, 15:27:22, 15:27:22, 15:27:40, 15:27...","[1443423302, 1443422479, 1443422141, 144342210...","[15, 15, 15, 15, 15, 15, 15]",3352c133a2ee0a29ed3351ab69dae032,16706.0,...,0,0,0,0,0,0,0,0,0,0
16760_b210865acfd2674ae0017b11271afc33,"[3230192, 3230175, 3230173, 3230168, 3230123, ...","[out-long, in-long, 2f-long, 1f-short, 2f-left...","[1448093621, 1448093653, 1448093653, 144809365...","[1386, 1354, 1056, 23, 659, 338]","[17, 17, 17, 17, 17, 17]","[17:13:41, 17:14:13, 17:14:13, 17:14:17, 17:16...","[1448095007, 1448095007, 1448094709, 144809368...","[17, 17, 17, 17, 17, 17]",b210865acfd2674ae0017b11271afc33,16760.0,...,0,0,0,0,0,0,0,0,0,0
16866_ec0ff57c50c68919aa1b5d3bd9424028,"[2022582, 2021645, 2021319, 2021277, 2021276]","[out-long, out-medium, out-medium, in-short, 1...","[1457251715, 1457254772, 1457255863, 145725598...","[2087, 233, 211, 21, 21]","[17, 17, 18, 18, 18]","[17:08:35, 17:59:32, 18:17:43, 18:19:47, 18:19...","[1457253802, 1457255005, 1457256074, 145725600...","[17, 18, 18, 18, 18]",ec0ff57c50c68919aa1b5d3bd9424028,16866.0,...,0,0,0,0,0,0,0,0,0,0
16724_8eb18e0cbf9c29ea1e5ed4f98cb05171,"[3632032, 3632031, 3632030, 3632029, 3631947, ...","[out-long, in-long, 1f-medium, 1f-right-medium...","[1444959503, 1444959503, 1444959503, 144495950...","[1094, 1094, 254, 203, 768, 29, 3492, 2807, 13...","[10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 1...","[10:38:23, 10:38:23, 10:38:23, 10:38:23, 10:43...","[1444960597, 1444960597, 1444959757, 144495970...","[10, 10, 10, 10, 10, 10, 12, 12, 11, 12, 11, 1...",8eb18e0cbf9c29ea1e5ed4f98cb05171,16724.0,...,0,0,0,0,0,0,0,0,0,0
16798_20ee02428cacb74eee06f0a155933689,"[2785691, 2785687, 2785674, 2785673]","[out-medium, in-medium, 2f-left-short, 2f-medium]","[1451370166, 1451370